<a href="https://colab.research.google.com/github/aleylani/Python/blob/main/Lecture_notes/Lec17_OOP_dataclasses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Lecture notes - OOP dataclasses

---
This is the lecture note for **OOP dataclass** - but it's built upon contents from previous lectures such as:
- input-output
- variables
- if-statement
- for loop
- while
- lists
- random
- strings
- functions
- error handling
- file handling
- dictionary
- OOP

<p class = "alert alert-info" role="alert"><b>Note</b> that this lecture note gives some introduction to OOP dataclasses.


Read more
- [dataclasses - Python docs](https://docs.python.org/3/library/dataclasses.html)
- [mutable default arguments - Hitchhikers guide to Python](https://docs.python-guide.org/writing/gotchas/#mutable-default-arguments)

---

## Dataclass

Dataclass gives some boilerplate code for free, for example
- `__init__()`
- `__repr__()`
- `__eq__()`
- and more ...

However if you need to customize them, you can override them. Dataclass are very useful for classes that mainly stores data. Note that dataclass started from Python 3.7 so older versions of Python doesn't support them.

In [ ]:
from dataclasses import dataclass
from __future__ import annotations


@dataclass
class Prefix:
    # fields - variable followed by type annotation
    # fields will be found by dataclass function

    value: int | float  # positional argument
    unit: str = "unit"  # positional argument
    prefix_symbol: str = None  # keyword argument

    # these are class attrbutes and not in the __init__
    # dataclass only looks for fields to create __repr__ and __init__
    prefix_symbols = "T G M k h d c m u n p".split()
    prefix_names = "tera giga mega kilo hekto deci centi milli mikro nano piko".split()
    prefix_values = (10 ** i for i in (12, 9, 6, 3, 2, -1, -2, -3, -6, -9, -12))

    prefix_dict = {
        symbol: [value, name]
        for name, symbol, value, in zip(prefix_names, prefix_symbols, prefix_values)
    }

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, value):
        if not isinstance(value, (float, int)):
            raise TypeError(f"value must be int or float not {type(value).__name__}")
        self._value = value

    def convert(self, prefix_symbol: str) -> float | int:
        self.prefix_symbol = prefix_symbol
        return self.value / self.prefix_dict[prefix_symbol][0]

    def __str__(self):
        if self.prefix_symbol:
            return f"{self.convert(self.prefix_symbol)}{self.prefix_symbol}{self.unit}"
        return f"{self.value} units"


p1 = Prefix(42, "g")  # this __init__ is generated by dataclass
print(p1)

p1.convert("m")

print(p1)
p2 = Prefix(231)
print(repr(p2))  # this __repr__ is generated by dataclass
print(p2)


## Dataclass with default mutable value

There is an antipattern in Python that can be very buggy if using mutable objects such as lists in default keyword argument in a function. This is because the mutable list is initiated one time, and not each time the function is called. For dataclass, it gives an exception when trying to create mutable default value. To solve this you must use something called a default_factory.

In [ ]:
from typing import List
from dataclasses import field, dataclass


@dataclass
class Person:
    name: str
    age: int


@dataclass
class Student(Person):
    language: str

    def __str__(self):
        return f"{self.name}, {self.age} år pratar {self.language}"


students_AI = {"Ada": 32, "Beda": 18, "Ceda": 30, "Deda": 21}


@dataclass
class AI:
    students: List[Student] = field(
        default_factory=lambda: [
            Student(name, age, "Python") for name, age in students_AI.items()
        ]
    )

    def __repr__(self) -> str:
        students = "\n".join(f"{student!s}" for student in self.students)
        return f"{'-'*30}\n{self.__class__.__name__}-klass:\n{students}\n{'-'*30}"


students_java = {"Eda": 12, "Geda": 28, "Heda": 40, "Koda": 31}


@dataclass
class Java:
    students: List[Student] = field(
        default_factory=lambda: [
            Student(name, age, "Java") for name, age in students_java.items()
        ]
    )

    def __repr__(self) -> str:
        students = "\n".join(f"{student!s}" for student in self.students)
        return f"{'-'*30}\n{self.__class__.__name__}-klass:\n{students}\n{'-'*30}"


AI22 = AI()

print(AI22)
Java()
